In [ ]:
import requests
import pandas as pd
import time
import base64
import concurrent.futures
import xml.etree.ElementTree as ET
import markdown
import re
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from sqlalchemy.types import Text, String, DateTime


kipris_key = "L/Ik7Sh0vMzeH5204ZB4D1pIFAdCplRSS2BqNpQvn8w="

headers = {"Authorization":f"Bearer {git_token}",
           "Accept": "application/vnd.github+json"}

keyword = ["robotics", "ROS", "robot arm", "robot", "amr"]
# keyword = ["ROS"]

delay = 2

rds_user = "root"
rds_pwd = "ckehdgus9v99nw!"
rds_host = "database-1.ctai4u0ayj03.ap-northeast-2.rds.amazonaws.com"
rds_port = 3306
rds_db = "amrbase"

repo_table_name = "github_repo"
read_table_name = "github_readmes"
kipris_table_name = "kipris_patent"

res = requests.get("https://api.github.com/user", headers=headers)

Kipris_serch_url = "http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice/getAdvancedSearch"

per_page = 50
max_page = 3

Default_total_target = 1000
Max_row_per_page = 500

In [48]:
def search_repos(keyword, language=None):
    repos = []
    for page in range(1, max_page + 1):
        query = keyword
        if language:
            query += f" language:{language}"
        url = "https://api.github.com/search/repositories"
        params = {"q": f"{query} stars:>=100",
                  "sort": "stars",
                  "order": "desc",
                  "per_page": per_page,
                  "page": page}
        r = requests.get(url, headers=headers, params=params)
        
        if r.status_code != 200:
            print(f"[Warning] 요청 실패 {r.status_code}")
            break
        
        batch = r.json().get("items", [])
        
        if not batch:
            break

        for repo in batch:
            desc = repo.get("description") or ""
            if not re.search(r"[\u4e00-\u9fff]", desc):
                repos.append(repo)
        # time.sleep(delay)
    return repos

In [49]:
def get_readme(owner, repo, token=None):
    url = f"https://api.github.com/repos/{owner}/{repo}/readme"
    headers = {"Accept": "application/vnd.github.v3+json"}
    if token :
        headers["Authorization"] = f"token {token}"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        data = response.json()
        content = base64.b64decode(data["content"].encode("utf-8")).decode("utf-8", errors="ignore")
        
        try:
            html = markdown.markdown(content)
            text = BeautifulSoup(html, "html.parser").get_text(separator="\n")
            return text.strip()
        except Exception as e:
            print(f"{owner}/{repo}: Markdown -> Text 변환 실패 {e})")
            return content
        
    except requests.exceptions.HTTPError as e:
        print(f"{owner}/{repo}: README 없음 ({response.status_code})")
        return "None"
    
    except Exception as e:
        print(f"{owner}/{repo} README 디코드 실패: {e}")
        return "None"

In [50]:
def fetch_repo_stats(repo, token):
    owner, name = repo["owner"]["login"], repo["name"]
    # stats = get_repo_stats(owner, name)
    readme = get_readme(owner, name, token)
    return {
        "full_name": repo["full_name"],
        # "description": repo["description"],
        "stars": repo["stargazers_count"],
        "forks": repo["forks_count"],
        "language": repo["language"],
        # "html_url": repo["html_url"],
        "updated_at": repo["updated_at"],
        # "commits": stats["commits"],
        # "contributors": stats["contributors"],
        # "open_issues": stats["open_issues"],
        "readme": readme
    }

In [51]:
engine_url = f"mysql+pymysql://{rds_user}:{rds_pwd}@{rds_host}:{rds_port}/{rds_db}"
engine = create_engine(engine_url)

In [52]:
all_repos = []
for kw in keyword:
    print(f"Searching keyword: {kw}")
    result = search_repos(kw)
    all_repos.extend(result)

# full_name 기준 중복 제거
unique_repos = {repo["full_name"]: repo for repo in all_repos}
repos_list = list(unique_repos.values())
print(f"총 {len(repos_list)}개의 고유 레포 수집 완료")

Searching keyword: robotics
Searching keyword: ROS
Searching keyword: robot arm
Searching keyword: robot
Searching keyword: amr
총 410개의 고유 레포 수집 완료


In [53]:
repo_data = []
total_repos = len(repos_list)

for i, repo in enumerate(repos_list, start=1):
    try:
        result = fetch_repo_stats(repo, git_token)
        if result:
            repo_data.append(result)
            print(f"[{i}/{total_repos}] {result['full_name']} 완료")
    except Exception as e:
        print(f"[{i}/{total_repos}] 오류 발생: {e}")
        continue

print("모든 README 수집 완료")

# DataFrame 생성
all_df = pd.DataFrame(repo_data)

df_meta = all_df[["full_name", "stars", "forks", "language", "updated_at"]]
df_read = all_df[["full_name", "readme"]]

[1/410] Developer-Y/cs-video-courses 완료
[2/410] commaai/openpilot 완료
[3/410] mudler/LocalAI 완료
[4/410] Genesis-Embodied-AI/Genesis 완료
[5/410] AtsushiSakai/PythonRobotics 완료
[6/410] wechaty/wechaty 완료
[7/410] huggingface/lerobot 완료
[8/410] hubotio/hubot 완료
[9/410] zauberzeug/nicegui 완료
[10/410] ArduPilot/ardupilot 완료
[11/410] bulletphysics/bullet3 완료
[12/410] rwaldron/johnny-five 완료
[13/410] DLR-RM/stable-baselines3 완료
[14/410] google-deepmind/mujoco 완료
[15/410] kornia/kornia 완료
[16/410] mozilla/TTS 완료
[17/410] rerun-io/rerun 완료
[18/410] hybridgroup/gobot 완료
[19/410] nasa-jpl/open-source-rover 완료
[20/410] StockSharp/StockSharp 완료
[21/410] dusty-nv/jetson-inference 완료
[22/410] Hypfer/Valetudo 완료
[23/410] cartographer-project/cartographer 완료
[24/410] firerpa/lamda 완료
[25/410] pliang279/awesome-multimodal-ml 완료
[26/410] jason718/awesome-self-supervised-learning 완료
[27/410] ClemensElflein/OpenMower 완료
[28/410] paperswithcode/ai-deadlines 완료
[29/410] serialport/node-serialport 완료
[30/410] ki

In [54]:
df_meta.to_sql(repo_table_name, con=engine, if_exists="replace", index=False, chunksize=100)
print(f"{repo_table_name} 테이블에 데이터 저장 완료")
df_read.to_sql(read_table_name, con=engine, if_exists="replace", index=False, chunksize=100)
print(f"{read_table_name} 테이블에 데이터 저장 완료")

github_repo 테이블에 데이터 저장 완료
github_readmes 테이블에 데이터 저장 완료


In [32]:
def serch_kipris(word_query,
                 service_key,
                 exclude_terms=None,
                 total_target=1000,
                 rows_per_page=500):
    K_result = []
    total_pages = (total_target + rows_per_page -1) // rows_per_page

    if exclude_terms is None:
        exclude_terms = []
    
    for page in range(1, total_pages + 1):
        kurl = (
            f"{Kipris_serch_url}"
            f"?word={word_query}"
            f"&numOfRows={rows_per_page}"
            f"&pageNo={page}"
            f"&ServiceKey={service_key}"
        )

        response = requests.get(kurl)
        response.raise_for_status()

        try:        
            root = ET.fromstring(response.content)
        except ET.ParseError:
            print(f"XML 파싱 실패: Page={page}, keyword={word_query}")
            continue
        
        items = root.findall(".//item")

        for item in items:
            inventionTitle = item.findtext("inventionTitle", default="").strip()
            astrtCont = item.findtext("astrtCont", default="").strip()

            if any(term.lower() in astrtCont.lower() or term.lower() in inventionTitle.lower() for term in exclude_terms):
                continue

            K_result.append({
                "inventionTitle":item.findtext("inventionTitle", default=""),
                "astrtCont":item.findtext("astrtCont", default="").strip(),
                "registerDate":item.findtext("registerDate", default=""),
                "applicationDate":item.findtext("applicationDate", default=""),
                "keyword":word_query
            })

        if len(items) < rows_per_page:
            break

        print(f"[Info] {page}페이지 완료 ({len(items)}건, 누적 {len(K_result)}건)")
        time.sleep(1.0)

    return K_result[:total_target]


In [33]:
def get_repo_stats(owner, repo_name):
    url = f"https://api.github.com/repos/{owner}/{repo_name}"
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        return {"commits": None, "contributors": None, "open_issues": None}
    repo = r.json()

    # 커밋 수 추정
    commits_url = f"https://api.github.com/repos/{owner}/{repo_name}/commits?per_page=1"
    commits_res = requests.get(commits_url, headers=headers)
    commits = None
    if "Link" in commits_res.headers:
        try:
            last_page = [l for l in commits_res.headers["Link"].split(",") if 'rel="last"' in l][0]
            commits = int(last_page.split("page=")[-1].split(">")[0])
        except:
            commits = None
    else:
        commits = len(commits_res.json())

    # 기여자 수 추정
    contrib_url = f"https://api.github.com/repos/{owner}/{repo_name}/contributors?per_page=1&anon=true"
    contrib_res = requests.get(contrib_url, headers=headers)
    contributors = None
    if "Link" in contrib_res.headers:
        try:
            last_page = [l for l in contrib_res.headers["Link"].split(",") if 'rel="last"' in l][0]
            contributors = int(last_page.split("page=")[-1].split(">")[0])
        except:
            contributors = None
    else:
        contributors = len(contrib_res.json())

    # 오픈 이슈 수
    open_issues = repo.get("open_issues_count", None)

    return {"commits": commits, "contributors": contributors, "open_issues": open_issues}

In [29]:
repos_list[62]

{'id': 102324163,
 'node_id': 'MDEwOlJlcG9zaXRvcnkxMDIzMjQxNjM=',
 'name': 'Ros',
 'full_name': 'Ewenwan/Ros',
 'private': False,
 'owner': {'login': 'Ewenwan',
  'id': 23008890,
  'node_id': 'MDQ6VXNlcjIzMDA4ODkw',
  'avatar_url': 'https://avatars.githubusercontent.com/u/23008890?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/Ewenwan',
  'html_url': 'https://github.com/Ewenwan',
  'followers_url': 'https://api.github.com/users/Ewenwan/followers',
  'following_url': 'https://api.github.com/users/Ewenwan/following{/other_user}',
  'gists_url': 'https://api.github.com/users/Ewenwan/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/Ewenwan/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/Ewenwan/subscriptions',
  'organizations_url': 'https://api.github.com/users/Ewenwan/orgs',
  'repos_url': 'https://api.github.com/users/Ewenwan/repos',
  'events_url': 'https://api.github.com/users/Ewenwan/events{/privacy}',
  'received_event

In [ ]:
# repo_data = []
# total_repos = len(repos_list)
# with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#     # 각 레포에 fetch_repo_stats를 병렬 실행
#     # results = list(executor.map(fetch_repo_stats, repos_list, [git_token]*len(repos_list)))

#     futures = [
#         executor.submit(fetch_repo_stats, repo, git_token)
#         for repo in repos_list
#     ]

#     for i, f in enumerate(concurrent.futures.as_completed(futures), start=1):
#         try:
#             result = f.result()
#             if result:
#                 repo_data.append(result)
#                 print(f"[{i}/{total_repos}] {result['full_name']} 완료")
#         except Exception as e:
#             print(f"[{i}/{total_repos}] Future 진행 중 오류 발생: {e}")
#             continue
            
#         if i >= total_repos - 1:
#             print("✅ 모든 README 수집 완료")
#             break

# # repo_data.extend(results)
# all_df = pd.DataFrame(repo_data)

# df_meta = all_df[["full_name", "stars", "forks", "language", "updated_at"]]
# df_read = all_df[["full_name", "readme"]]

In [ ]:
all_patent = []
exclude_term = ["화학", "반응성", "화합", "제어능", "활성", "미토콘트리아", "단백질", "치료", "약물", "효소"]

for kw in keyword:
    print(f"Searching keyword: {kw}")
    result = serch_kipris(kw, kipris_key, exclude_term)
    all_patent.extend(result)
    print(f"{kw} 완료 (누적 {len(all_patent)}건)")

kdf = pd.DataFrame(all_patent, columns=["inventionTitle", "astrtCont", "registerDate", "applicationDate","keyword"])

kdf.to_sql(kipris_table_name,
           con=engine,
           if_exists="replace",
           index=False,
           chunksize=100,
           dtype={
               "inventionTitle": String(500),
               "astrtCont": Text,
               "registerDate": String(20),
               "keyword": String(100)
           })
print(f"{kipris_table_name} 테이블에 데이터 저장 완료")

Searching keyword: ROS
[Info] 1페이지 완료 (500건, 누적 160건)
[Info] 2페이지 완료 (500건, 누적 233건)
kipris_patent 테이블에 데이터 저장 완료


In [55]:
df_meta.to_csv("~/github_repo.csv", index=False)
df_read.to_csv("~/github_readmes.csv", index=False)

In [ ]:
kdf.to_csv("~/devkipris_patent.csv", index=False)

In [44]:
df_read

,full_name,readme
0,dotnet/roslyn,The .NET Compiler Platform\n\n\n\n\nRoslyn is ...
1,ArduPilot/ardupilot,ArduPilot Project\n\n\n\n\n \n \n \n \n\n\n \n...
2,carla-simulator/carla,CARLA Simulator\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nC...
3,autowarefoundation/autoware,Autoware - the world's leading open-source sof...
4,PX4/PX4-Autopilot,PX4 Drone Autopilot\n\n\n \n\n\n \n\n\n\n\nThi...
...,...,...
137,Filsommer/rosencharts,Rosen - A React Charting Library\n\n\nA lightw...
138,ntnu-arl/gbplanner_ros,Graph-based Exploration Planner 2.0\n\n\n\n\nT...
139,SteveMacenski/spatio_temporal_voxel_layer,Spatio-Temporal Voxel Layer\n\n\nThis is a dro...
140,baker-laboratory/RoseTTAFold-All-Atom,Code for RoseTTAFold All-Atom\n\n\n\n\n\n\n\n\...
